Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

c:\jonescode\PINN-capstone\icepinn.py:338: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if tenths_remaining is not 0:


cuda
1
cuda


In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1
#NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [3]:
# Define model attributes; instantiate model
model_dimensions = torch.tensor([8, 8]).to(device)
is_sf_PINN = torch.tensor(False)
model1 = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[1],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model1.register_buffer('dimensions', model_dimensions)
model1.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
model1.apply(ip.init_HE)

optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-5)

# Define learning rate scheduling scheme
scheduler_summed = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10000
    )

In [4]:
MODEL_NAME = "HE_8wide_nodiff"
print(MODEL_NAME)
print(training_set.shape)
print(ip.calc_cp_loss(model1, training_set, ip.get_misc_params(), 10, 1, 1, False, False).shape)
print(ip.enforced_model(training_set, model1).shape)
print(model1)

HE_8wide_nodiff
torch.Size([1920, 2])
torch.Size([1920, 2])
torch.Size([1920, 2])
IcePINN(
  (sml): SinusoidalMappingLayer()
  (post_sml): Linear(in_features=24, out_features=8, bias=True)
  (sin): SinActivation()
  (fc_in): Linear(in_features=2, out_features=8, bias=True)
  (post_fc_in): Linear(in_features=8, out_features=8, bias=True)
  (fc_hidden): ModuleList(
    (0-5): 6 x Linear(in_features=8, out_features=8, bias=True)
  )
  (fc_out): Linear(in_features=8, out_features=2, bias=True)
)


Train model

In [5]:
ip.train_IcePINN(
    model=model1, 
    optimizer=optimizer, 
    training_set=training_set, 
    epochs=100_000, 
    name=MODEL_NAME, 
    print_every=1_000, 
    print_gradients=False,
    diffusion=False,
    LR_scheduler=None)

Commencing PINN training on 1920 points for 100000 epochs.
Epoch [1k/100k] at 0m 10s: Ntot = 3100.481, Nqll = 4859.067, LR = 1e-05
Epoch [2k/100k] at 0m 21s: Ntot = 1687.048, Nqll = 1608.094, LR = 1e-05
Epoch [3k/100k] at 0m 30s: Ntot = 1514.784, Nqll = 972.930, LR = 1e-05
Epoch [4k/100k] at 0m 40s: Ntot = 1432.528, Nqll = 546.141, LR = 1e-05
Epoch [5k/100k] at 0m 51s: Ntot = 1401.759, Nqll = 341.780, LR = 1e-05
Epoch [6k/100k] at 1m 2s: Ntot = 1398.842, Nqll = 263.878, LR = 1e-05
Epoch [7k/100k] at 1m 12s: Ntot = 1400.294, Nqll = 237.385, LR = 1e-05
Epoch [8k/100k] at 1m 22s: Ntot = 1400.903, Nqll = 226.740, LR = 1e-05
Epoch [9k/100k] at 1m 32s: Ntot = 1401.767, Nqll = 219.552, LR = 1e-05
Epoch [10k/100k] at 1m 42s: Ntot = 1402.802, Nqll = 213.335, LR = 1e-05
Training 1/10 complete! Time remaining estimate: 15m 18s
Epoch [11k/100k] at 1m 52s: Ntot = 1404.143, Nqll = 207.777, LR = 1e-05
Epoch [12k/100k] at 2m 3s: Ntot = 1406.008, Nqll = 202.589, LR = 1e-05
Epoch [13k/100k] at 2m 14s: N

In [ ]:
# Instantiate model2 as best version of model1
# model2 = ip.load_IcePINN(MODEL_NAME)
# model2.train()
# optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-6)

Train again on smaller LR starting from saved checkpoint (best saved model above)

In [ ]:
# ip.train_IcePINN(
#     model=model2, 
#     optimizer=optimizer2, 
#     training_set=training_set, 
#     epochs=100_000, 
#     name=MODEL_NAME+"_round2", 
#     print_every=1_000, 
#     print_gradients=False,
#     diffusion=True,
#     LR_scheduler=None)